In [10]:
import os
import glob
import random
import cv2
import numpy as np
import multiprocessing
from functools import partial

In [11]:
image_folder = r"C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr/*.jpg"
files = glob.glob(image_folder)

In [12]:
def process_image(filename, width, height):
    img = cv2.imread(filename)
    if img is None:  # Check if the file could be opened
        print("Image failed to load :", filename)
        return None
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (width, height), None)
    # If found, add object points, image points (after refining them)
    if ret:
        # Termination criteria for finding the sub pixel coordinates of corners (cornerSubPix)
        criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 60, 0.001)
        # Increase the pixel location accuracy
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        return corners2
    print("No chessboard : ", filename)
    os.remove(filename)
    return None

In [13]:
# internal corners of the grid
width, height = 6, 4
used_files = []
imgpoints = []  # 2d points in image plane
for f in files:
    pixel_points = process_image(f, width=width, height=height)
    if pixel_points is not None:
        used_files.append(f)
        imgpoints.append(pixel_points)
len(imgpoints)

127

In [14]:
# We assume the checker pattern is kept in the Z=0 plane and the camera is moved and rotated
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((height*width, 3), np.float32)
objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

# Scale the 3d points to the actual dimensinos
squareSize = 36.5e-3  # meter
# objp = objp*squareSize
objpoints = []  # 3d point in  world space
[objpoints.append(objp) for i in range(len(imgpoints))]
len(objpoints)

127

In [15]:
runs = 1
mtx_sum = np.zeros((3,3))
dist_sum = np.zeros((1,5))
for r in range(runs):
    batch = 127
    idxs = random.sample(list(range(len(objpoints))), batch)
    batch_obj, batch_img = [], []
    for i in idxs:
        batch_obj.append(objpoints[i])
        batch_img.append(imgpoints[i])
    img = cv2.imread(files[0])
    rms, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(batch_obj, batch_img, img.shape[1::-1], None, None)
    mtx_sum += mtx
    dist_sum += dist
    print("RMS:", rms)
    # print("Camera Matrix:\n", mtx)
    # print("Distortion Coefficients:\n", dist.ravel())

RMS: 0.5094508815623193


In [16]:
mtx = mtx_sum/runs
dist = dist_sum/runs
print("Camera Matrix:\n", mtx)
print("Distortion Coefficients:\n", dist.ravel())

Camera Matrix:
 [[431.87822876   0.         433.16900824]
 [  0.         429.71615176 241.94768897]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [-0.2166708   0.05382875  0.00065365  0.00136901 -0.00666539]


In [17]:
errors = np.zeros(batch)
for i in range(len(rvecs)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    errors[i] = error

In [18]:
for i in reversed(np.argsort(errors)):
    full_idx = idxs[i]
    print(files[full_idx], errors[i])

C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\2802.jpg 98.74679183871389
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\1759.jpg 94.29656015556913
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\1237.jpg 74.19486637815584
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\1417.jpg 73.84593485211686
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\1819.jpg 69.23196182765811
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\2209.jpg 66.90780631043681
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\1130.jpg 65.88019540051518
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_1080_w_lr\2798.jpg 65.76936088470605
C:\Users\lukeasargen\projects\uas_misc\cam_calibration\images\gopro_session_5_10